In [1]:
import pandas as pd
from pyproj import Proj, transform
import fiona
from fiona.crs import from_epsg
import logging
import sys


# outProj = Proj(init='epsg:3857')
# inProj = Proj(init='epsg:4326') # It is equal to WSG

In [2]:
shape = fiona.open("mp/Uso_das_Terras_e_Cobertura_Vegetal__1999.shp")
print (shape.schema)
# Chico Mendes é o id 73

{'properties': OrderedDict([('OBJECTID', 'int:4'), ('VEGETACAO', 'str:17'), ('RESUMO', 'str:10'), ('A99', 'float:24.15'), ('RuleID', 'str:17')]), 'geometry': 'Polygon'}


In [3]:
shape[1696]

{'type': 'Feature',
 'id': '1696',
 'properties': OrderedDict([('OBJECTID', 1697),
              ('VEGETACAO', 'Área Urbana'),
              ('RESUMO', 'Artificial'),
              ('A99', 9337100.29492188),
              ('RuleID', 'Área Urbana')]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(661887.1616000002, 7455420.579),
    (661852.2241000002, 7455365.5789),
    (661756.4113999996, 7455338.0789),
    (661634.4737, 7455269.5789),
    (661667.4737999998, 7455224.5788),
    (661654.2237, 7455071.0786),
    (661393.8483999996, 7455040.0786),
    (661466.6608999996, 7454545.5779),
    (661526.5983999996, 7454524.5779),
    (661518.8483999996, 7454452.5778),
    (661502.7233999996, 7454471.5778),
    (661463.9732999997, 7454459.0778),
    (661425.2232999997, 7454427.5778),
    (661412.2857999997, 7454390.0777),
    (661380.0356999999, 7454346.0777),
    (661339.1606999999, 7454346.5777),
    (661288.0356000001, 7454331.5777),
    (661235.5980000002, 7454338.0777),
    (661199.

In [4]:
# Transformar escala lag, long em Metros no globo achatado

# salvando o esquema e pegando a projecao de entrada
sink_schema = shape.schema.copy()
p_in = Proj(shape.crs)

# definindo um novo shape de saida, no mesmo local do shape atual com a projeção mudada
with fiona.open(
        'area_1999.shp', 'w',
        crs=from_epsg(3857),
        driver=shape.driver,
        schema=sink_schema,
        ) as sink:

    p_out = Proj(sink.crs)

    # fazendo apenas para o Chico Mendes (esse arquivo tem Polygon e MultiPolygon)
    # nesse caso, se fizesse para o arquivo todo, nao seria contemplado alguns shapes
    f = shape[1696]

    try:
        # verifica se eh da estrutura poligono
        assert f['geometry']['type'] == "Polygon"
        new_coords = []
        # para todas as coordenadas do poligono
        for ring in f['geometry']['coordinates']:
            x2, y2 = transform(p_in, p_out, *zip(*ring)) #transforma
            new_coords.append(zip(x2, y2)) # salva a nova lista de coordenadas
        f['geometry']['coordinates'] = new_coords
        sink.write(f)

    except Exception:
        # Writing uncleanable features to a different shapefile
        # is another option.
        logging.exception("Error transforming feature %s:", f['id'])

/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [5]:
shape2 = fiona.open("area_1999.shp")
print (shape2.schema)

{'properties': OrderedDict([('OBJECTID', 'int:4'), ('VEGETACAO', 'str:17'), ('RESUMO', 'str:10'), ('A99', 'float:24.15'), ('RuleID', 'str:17')]), 'geometry': 'Polygon'}


In [6]:
shape2[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('OBJECTID', 1697),
              ('VEGETACAO', 'Área Urbana'),
              ('RESUMO', 'Artificial'),
              ('A99', 9337100.29492188),
              ('RuleID', 'Área Urbana')]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-4833542.228408624, -2632223.5114251063),
    (-4833579.521503947, -2632283.984866917),
    (-4833683.237529001, -2632315.1429879298),
    (-4833814.84276101, -2632391.3815495507),
    (-4833778.483733997, -2632440.1362530086),
    (-4833791.077745259, -2632607.923384899),
    (-4834073.448694694, -2632644.8350308156),
    (-4833988.611277128, -2633184.018044371),
    (-4833923.279645957, -2633206.2483416772),
    (-4833930.854509134, -2633284.9712624834),
    (-4833948.586701053, -2633264.4105674),
    (-4833990.519852203, -2633278.517000616),
    (-4834032.231190887, -2633313.373474531),
    (-4834045.842488683, -2633354.4797472497),
    (-4834080.34990371, -2633402.911455627),
    (-4834